In [1]:
import logging
import pandas as pd
import os
import json

In [2]:
from retrieve_data_by_query import RetrieveDataByQuery
# t = RetrieveDataByQuery(sessionid="65955050144%3AsDI3oarlmsipI4%3A10%3AAYfZrHTizKd733ih8H3yTtTARkNxla7Jr5Fe_C14Iw")
# t = RetrieveDataByQuery(sessionid="67725660272%3AW741uU0WEqnpeU%3A11%3AAYc8uSJT5zhhAmFGrTsolApwMbEuOfyAr5P4hqX8IA")
# t = RetrieveDataByQuery(sessionid="72311449442%3AlUryE0KV76UBLs%3A27%3AAYcrmho9upEOXJN1UXfXFAcPBrxEthfQark1yarqDg")
# t = RetrieveDataByQuery(sessionid="72164089558%3ArAwUfcPDA9EFBe%3A11%3AAYfCrETIzCPGInwfJ21wGhSlRwbbdWTecG10Kd5GaA")
# t = RetrieveDataByQuery(sessionid="72164089558%3As83zYF4EnDh0UB%3A9%3AAYeSZtx6U69boU7q1yS7sXUdDgCwWuzZP-LYlG2hDg")
t = RetrieveDataByQuery(sessionid="72311449442%3A06XiI9UcgYrQNj%3A28%3AAYff3Vm7a8Xdcq5D3KpeACcLc9fiw9ums2V_FsU0qg")
# t.retrieve_follower_by_userid_after("62231305945", str(25 * 1))

In [3]:
query = "Artificial Intelligence"

In [4]:
def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open("progress2.json", "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open("progress2.json", "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        if batch_number == 1:
            response = t.retrieve_follower_by_userid(userid)
            data = response.get("data", {}).get("user", {}).get("followers", {})
            d_out = get_follower_data(data.get("edges", []))
            for res in d_out:
                allpk.append(res["pk"])
            results.extend(d_out)
            has_next_page = data.get("page_info", {}).get("has_next_page", False)
        else:
            has_next_page = True

        while has_next_page:
            try:
                print("Pagina:", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results




def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [5]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [6]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [7]:
df = pd.read_csv("data_parte2.csv")
df['user_pk'] = df['user_pk'].astype(str)
unique_user_pk = df['user_pk'].unique()
len(unique_user_pk)

293

In [8]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_data2.csv"
processed_users = load_processed_users(csv_file)

In [9]:
def fetch_followers(pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")

Starting follower data collection
Skipping already processed user_pk: 192592722
Skipping already processed user_pk: 259533357
Skipping already processed user_pk: 387960
Skipping already processed user_pk: 60903176602
Skipping already processed user_pk: 69724808842
Skipping already processed user_pk: 5642593129
Skipping already processed user_pk: 25004361
Skipping already processed user_pk: 54052706
Skipping already processed user_pk: 30628914796
Skipping already processed user_pk: 11630601130
Skipping already processed user_pk: 18371369
Skipping already processed user_pk: 60837322123
Skipping already processed user_pk: 25058521708
Skipping already processed user_pk: 508545432
Skipping already processed user_pk: 1502777818
Skipping already processed user_pk: 40472391
Skipping already processed user_pk: 2267199523
Skipping already processed user_pk: 49573589494
Skipping already processed user_pk: 53223130121
Skipping already processed user_pk: 22181915186
Skipping already processed user_

In [10]:
df1 = pd.read_csv("followers_data2.csv")
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(293, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
33762,725,725,725
129231,3002,3002,3002
244284,49,49,49
265121,49,49,49
374099,49,49,49
...,...,...,...
68126431458,1337,1337,1337
69724808842,497,497,497
70995803294,365,365,365


In [11]:
df1 = pd.read_csv("followers_data2.csv")
df1 = df1.drop_duplicates()
df1
df1.to_csv("followers_data2.csv")